In [ ]:
import fitz  # PyMuPDF

def compress_pdf(input_pdf_path, output_pdf_path, zoom=2, quality=60):
    # Open the input PDF
    pdf_document = fitz.open(input_pdf_path)
    pdf_writer = fitz.open()

    # Iterate through each page
    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num)
        pix = page.get_pixmap(matrix=fitz.Matrix(zoom, zoom), alpha=False)

        # Convert to PIL image for compression
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

        # Compress the image
        compressed_image_io = io.BytesIO()
        img.save(compressed_image_io, format='JPEG', quality=quality)

        # Create a new PDF page with the compressed image
        new_page = pdf_writer.new_page(width=pix.width, height=pix.height)
        new_page.insert_image(new_page.rect, stream=compressed_image_io.getvalue())

    # Save the compressed PDF
    pdf_writer.save(output_pdf_path)
    pdf_writer.close()

# Usage
input_pdf = '/content/NRI DOCUMENT.pdf'
output_pdf = 'output_compressed1.pdf'
compress_pdf(input_pdf, output_pdf, zoom=1, quality=100)


input_pdf_size_kb = os.path.getsize(input_pdf) / 1024
print(input_pdf_size_kb)
pdf_size_kb = os.path.getsize(output_pdf) / 1024
print(pdf_size_kb)


696.955078125
11332.2158203125


In [ ]:
import os
from pypdf import PdfWriter, PdfReader
from PIL import Image
import io

def compress_image(image_data, quality=50):
    """Compress the image and return as a byte stream."""
    image = Image.open(io.BytesIO(image_data))
    with io.BytesIO() as output:
        image.save(output, format="JPEG", quality=quality, optimize=True)
        return output.getvalue()


def compress_images_in_pdf(input_pdf, output_pdf, image_quality=50):
    reader = PdfReader(input_pdf)
    writer = PdfWriter()

    for page in reader.pages:
        if "/XObject" in page["/Resources"]:
            xobject = page["/Resources"]["/XObject"].get_object()
            for obj in xobject:
                if xobject[obj]["/Subtype"] == "/Image":
                    original_data = xobject[obj]._data
                    compressed_data = compress_image(original_data, quality=image_quality)
                    xobject[obj]._data = compressed_data
        writer.add_page(page)

    with open(output_pdf, "wb") as f:
        writer.write(f)

# File paths
input_pdf = "/content/out3.pdf"
output_pdf = "compressed_images_nri.pdf"
final_pdf = "final_nri_compressed.pdf"

# Get the size of the original PDF
original_size = os.path.getsize(input_pdf)

# Compress images within the PDF
compress_images_in_pdf(input_pdf, output_pdf, image_quality=10)

# Compress the PDF content streams
writer = PdfWriter(clone_from=output_pdf)

for page in writer.pages:
    page.compress_content_streams(level=9)  # Maximum compression

# Write the fully compressed PDF
with open(final_pdf, "wb") as f:
    writer.write(f)

# Get the size of the final compressed PDF
compressed_size = os.path.getsize(final_pdf)

# Print the results
print(f"Original PDF size: {original_size} bytes")
print(f"Compressed PDF size (images + PDF): {compressed_size} bytes")
print(f"Size reduction: {original_size - compressed_size} bytes ({(original_size - compressed_size) / original_size * 100:.2f}% reduction)")


UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7a4160d5d850>

In [ ]:
from pdf2image import convert_from_path
import os

def pdf_to_images(input_pdf, output_folder, dpi=300):
    """
    Convert each page of the PDF to an image and save them.

    :param input_pdf: Path to the input PDF file.
    :param output_folder: Folder to save the output images.
    :param dpi: Resolution for the output images (default is 300).
    """
    # Convert PDF to a list of images
    images = convert_from_path(input_pdf, dpi=dpi)

    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Save each image
    for i, image in enumerate(images):
        image_path = os.path.join(output_folder, f"page_{i+1}.png")
        image.save(image_path, "PNG")
        print(f"Saved {image_path}")

def get_folder_size(folder):
    """
    Calculate the total size of all files in a folder.

    :param folder: Path to the folder.
    :return: Total size of the folder in bytes.
    """
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder):
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)
            total_size += os.path.getsize(file_path)
    return total_size

# Example usage
input_pdf = "/content/NRI DOCUMENT.pdf"
output_folder = "output_images"

# Convert PDF pages to images
pdf_to_images(input_pdf, output_folder, dpi=300)

# Calculate the total size of the output images folder
folder_size = get_folder_size(output_folder)

# Print the total size in bytes, KB, and MB
print(f"Total size of '{output_folder}' folder: {folder_size} bytes")
print(f"Total size of '{output_folder}' folder: {folder_size / 1024:.2f} KB")
print(f"Total size of '{output_folder}' folder: {folder_size / (1024 * 1024):.2f} MB")


Saved output_images/page_1.png
Saved output_images/page_2.png
Saved output_images/page_3.png
Saved output_images/page_4.png
Saved output_images/page_5.png
Saved output_images/page_6.png
Saved output_images/page_7.png
Saved output_images/page_8.png
Saved output_images/page_9.png
Saved output_images/page_10.png
Saved output_images/page_11.png
Saved output_images/page_12.png
Saved output_images/page_13.png
Saved output_images/page_14.png
Saved output_images/page_15.png
Saved output_images/page_16.png
Saved output_images/page_17.png
Saved output_images/page_18.png
Saved output_images/page_19.png
Saved output_images/page_20.png
Saved output_images/page_21.png
Saved output_images/page_22.png
Saved output_images/page_23.png
Saved output_images/page_24.png
Saved output_images/page_25.png
Saved output_images/page_26.png
Saved output_images/page_27.png
Saved output_images/page_28.png
Saved output_images/page_29.png
Saved output_images/page_30.png
Saved output_images/page_31.png
Saved output_imag

In [ ]:
from PIL import Image
from fpdf import FPDF
import os

def compress_image(image_path, output_path, quality=50):
    """
    Compress an image and save it.

    :param image_path: Path to the input image.
    :param output_path: Path to save the compressed image.
    :param quality: Compression quality (1-100).
    """
    image = Image.open(image_path)
    image.save(output_path, "JPEG", quality=quality)

def images_to_pdf(image_folder, output_pdf, target_size_kb=500):
    """
    Convert a folder of images to a PDF with a target size.

    :param image_folder: Path to the folder containing images.
    :param output_pdf: Path to save the output PDF.
    :param target_size_kb: Desired maximum size of the PDF in KB.
    """
    quality = 85  # Start with a high quality
    pdf = FPDF()
    image_paths = sorted([os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(".png")])

    while True:
        # Create a temporary folder for compressed images
        compressed_folder = os.path.join(image_folder, "compressed")
        os.makedirs(compressed_folder, exist_ok=True)

        # Compress images
        compressed_images = []
        for i, image_path in enumerate(image_paths):
            compressed_image_path = os.path.join(compressed_folder, f"compressed_{i+1}.jpg")
            compress_image(image_path, compressed_image_path, quality=quality)
            compressed_images.append(compressed_image_path)

        # Add images to PDF
        pdf = FPDF()
        for img in compressed_images:
            pdf.add_page()
            pdf.image(img, 0, 0, 210, 297)  # A4 size (210x297 mm)

        # Save the PDF
        pdf.output(output_pdf)

        # Check PDF size
        pdf_size_kb = os.path.getsize(output_pdf) / 1024
        print(pdf_size_kb)
        if pdf_size_kb <= target_size_kb:
            break
        else:
            quality -= 10  # Reduce quality for further compression

        # Clean up compressed images
        for img in compressed_images:
            os.remove(img)
        os.rmdir(compressed_folder)

        if quality < 1:
            print("Warning: Cannot compress PDF to target size with acceptable quality.")
            break

    print(f"Final PDF size: {pdf_size_kb:.2f} KB")

# Example usage
image_folder = "output_images"  # Folder where the images are stored
output_pdf = "output.pdf"

images_to_pdf(image_folder, output_pdf, target_size_kb=500)


34087.326171875
28213.9384765625
24970.70703125
22619.67578125
20918.4970703125
19064.830078125
16748.72265625
13746.798828125
9021.2333984375
Final PDF size: 9021.23 KB


In [ ]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=c425f449cf656839c816b111cbe5c4011f769220b01be7ee36ca1a577032fd8b
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf
